## Извлечение ключевых слов


In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [2]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [3]:
!pip install pymorphy2
!pip install pymystem3==0.1.10
!pip install rusenttokenize
!pip install razdel

     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 8.2MB 11.1MB/s 
  Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
from pymorphy2.tokenizers import simple_word_tokenize
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from razdel import tokenize as razdel_tokenize
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [6]:
import numpy as np

In [7]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [8]:
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_0.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_1.jsonlines.zip

--2021-06-25 10:43:27--  https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_0.jsonlines.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/ru_kw_eval_datasets/master/data/ng_0.jsonlines.zip [following]
--2021-06-25 10:43:27--  https://raw.githubusercontent.com/mannefedov/ru_kw_eval_datasets/master/data/ng_0.jsonlines.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2987145 (2.8M) [application/zip]
Saving to: ‘ng_0.jsonlines.zip.2’

ng_0.jsonlines.zip. 100%[===================>]   2.85M  --.-KB/s    in 0.1s    

2021-06-25 10:43:28 (24.0 MB/s) - ‘ng_0.jsonline

In [9]:
!unzip ng_0.jsonlines.zip 
!unzip ng_1.jsonlines.zip

Archive:  ng_0.jsonlines.zip
replace ng_0.jsonlines? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ng_0.jsonlines          
Archive:  ng_1.jsonlines.zip
replace ng_1.jsonlines? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ng_1.jsonlines          


In [11]:
PATH_TO_DATA = './'

In [15]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA) if file.endswith('jsonlines')]

Объединим файлы в один датасет.

In [16]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:5], axis=0, ignore_index=True)

In [17]:
data.shape

(1987, 5)

In [18]:
data.head(3)

,keywords,title,url,content,summary
0,"[школа, образовательные стандарты, литература, история, фгос]","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html,"В среду состоялось отложенное заседание Совета по федеральным государственным образовательным стандартам (ФГОС) при Министерстве образования и науки РФ. Собрание должно было состояться еще в понедельник, но было перенесено по просьбе членов совета. И вот пришло сообщение, что общественники выразили согласие с позицией министерства. Новые ФГОСы приняты.\nНа вчерашнем заседании был принят ФГОС по начальной общеобразовательной школе. До 28 марта продлятся косультации по ФГОСам для средней школы.\nНапомним, что накануне Гильдия словесников разместила открытое письмо на имя министра образования и науки РФ Ольги Васильевой. По мнению авторов письма, новые ФГОСы грубо нарушают права детей, уже проучившихся по существующему стандарту до 6-го класса. Приняв новый стандарт, Министерство образования дает право контролирующим органам ловить детей на незнании большого списка произведений (235 за пять лет обучения). «Это исключает возможность полноценного их освоения, создает риск формального, п...","Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов виноваты издательства учебной литературы"
1,"[красота, законы]",У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрения. Плохо было б, если б она вовсе не озаряла своим светом космическую темень пустоты зрачка. Слава богу, такое вряд ли возможно. \nА случается, что красота уходит. Почему вдруг? И куда она девается, когда в один из философских обходов своего организма вы, еще недавно гордый ее обладатель, обескураженно ее недосчитываетесь? \nВообразите: прелестнейшее из созданий – ваша кошка пластичнейшими движениями рвет банкноту за банкнотой, забирается на карниз по шелковой занавеске или отгрызает полпаспорта. Где, скажите, теперь красота этой кошки? Или другой пример – с зазнобой сердца. Предмет романтичнейших грез наконец-то садится с вами на заветную скамейку в парке – закат, пение птах… И тут он силой своего обаяния с оглушительным плюхом обрушивает вокруг вас красоту и гармонию столетних дубов, тополей и прочего. Где, спрашивается, красота момента? \nЕсли от сказки после того, как ее рассказали,...",О живительной пользе укорота при выборе между плохим и хорошим
2,"[юзефович, гражданская война, пепеляев, якутия]",Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html,"Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» – она стала интеллектуальным бестселлером и классикой жанра – документальный роман. В то время автор попутно изучал и историю вооруженного восстания в Якутии в 1922–1923 годах под руководством Анатолия Пепеляева. И вот теперь из «якутского» материала сложилась отдельная книга. Тема ее для нынешнего читателя поистине раритетна. Ведь воевавший где-то на самом краю страны Пепеляев практически забыт, притом что о борьбе с ним когда-то в СССР выходили статьи и книги. В памяти потомков, образно говоря, от Пепеляева остался только пепел.\nЮзефович воскрешает в памяти не только его военные дела, но и человеческие черты. Этот провинциальный интеллигент, неврастеник и фаталист, начал восстание, практически не имея шансов на успех. Однако силою недюжинной харизмы Пепеляев сумел собрать вокруг себя многих боевых офицеров, таежных охотников и недовольных новыми порядками аборигенов. Для своих 32 лет ...",Крепость из тел и призрак независимой Якутии


Каждой статье приписано какое-то количество ключевых слов. **Наша задача - придумать как извлекать точно такой же список автоматически.**
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [19]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [20]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Бейзлайн

In [23]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [25]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [26]:
data['content_norm'] = data['content'].apply(normalize)

In [27]:
data['title_norm'] = data['title'].apply(normalize)

In [28]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [29]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)

In [30]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=2, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [31]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [32]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [33]:
## так как матрица в tfidf в спарс формате, ее нельзя просто так отсортировать
## перевести ее в обычный формат для всех данных тоже не получится - не хватит памяти
## поэтому пройдем по строчкам, переведем строчку в обычный array и отсортируем ее
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [34]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Возьмем этот результат за **baseline.**

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09

# Подходы к преодолению бейзлайна

Посмотрим еще раз на то, что бывает в оригинальных ключевых словах.

In [39]:
list(data['keywords'])[:10]

[['школа', 'образовательные стандарты', 'литература', 'история', 'фгос'],
 ['красота', 'законы'],
 ['юзефович', 'гражданская война', 'пепеляев', 'якутия'],
 ['формула1', 'автоспорт', 'гонки', 'испания', 'квят'],
 ['есенин',
  'православие',
  'святая русь',
  'поэзия',
  'год литературы',
  'клюев',
  'мариенгоф',
  'стихи',
  'россия'],
 ['медвузы', 'медицинское образование', 'рудн', 'николай стуров', 'интервью'],
 ['литература',
  'книги',
  'периодика',
  'космос',
  'небо',
  'астрономия',
  'анатомия',
  'филология'],
 ['сша', 'ирак', 'война'],
 ['искусственный интеллект', 'робот', 'компьютер', 'технологии'],
 ['вб', 'вто', 'переговоры', 'тарифы']]

1. Можно отметить, что достаточно часто встречаются прилагательные. Попробуем их тоже оставлять при нормализации (и числительные тоже).

In [57]:
def normalize_2(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ['NOUN', 'ADJF', 'NUMR']]

    return words

In [58]:
data['content_norm_2'] = data['content'].apply(normalize_2)

In [59]:
data['title_norm_2'] = data['title'].apply(normalize_2)

In [60]:
data['content_norm_str_2'] = data['content_norm_2'].apply(' '.join)

In [61]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)

In [62]:
tfidf.fit(data['content_norm_str_2'])
texts_vectors = tfidf.transform(data['content_norm_str_2'])

In [63]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [64]:
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [65]:
evaluate(data['keywords'], keywords)

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09


Не вышло :'D

2. Попробуем оставить только униграммы, так как, кажется, биграмм не так много. Также немного увеличим значение min_df (с 2 до 5)

In [66]:
tfidf = TfidfVectorizer(ngram_range=(1,1), min_df=5)

In [67]:
tfidf.fit(data['content_norm_str'])
texts_vectors = tfidf.transform(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [68]:
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [69]:
evaluate(data['keywords'], keywords)

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


Бейзлайн побит.

3. Тут я пробовала поработать с параметрами gensim, но результат так и не побил бейзлайн

In [70]:
from gensim.summarization import keywords

In [77]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x), pos_filter=['NN']).split('\n')[:10])
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


4. Rake

In [81]:
!pip install rake-nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7829 sha256=3662c5b0e2ef5bcfaf3fa443f5f3b86d9474152d3619c811973956000e39679f
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [103]:
from rake_nltk import Rake

rake = Rake(language = 'russian', stopwords = stops, punctuations = punct)

In [104]:
keywords = []

for text in list(data['content_norm_str']):
    rake.extract_keywords_from_text(text)
    #print(len(rake.get_ranked_phrases()))
    kwords = [word for phrase in rake.get_ranked_phrases() for word in phrase.split(' ')]
    c = Counter(kwords)
    keywords.append([words for words, i in c.most_common(10)])

evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


Бейзлайн побит.

Я пробовала задавать и другие параметры, например, max_length, но тогда результат сильно ушудшался.

5. Графы без networkx

In [110]:
from itertools import combinations

In [137]:
def get_kws(text, top=5, window_size=5, random_p=0.1, len_check = False):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    
    if len_check == True:
      counter_dict = {k:v for k, v in c.items() if len(id2word[k]) > 5 and id2word[k] not in stops}
      c = Counter(counter_dict)
    
    # вернем топ-N
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n

In [112]:
from tqdm import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [113]:
%%time
keywords_rw = data['content_norm'].progress_apply(lambda x: get_kws(x, 10, 10))

100%|██████████| 1987/1987 [02:41<00:00, 12.31it/s]

CPU times: user 2min 42s, sys: 14.4 s, total: 2min 57s
Wall time: 2min 41s


In [114]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [119]:
%%time
keywords_rw = data['content_norm'].progress_apply(lambda x: get_kws(x, top = 10, window_size = 30))

100%|██████████| 1987/1987 [02:48<00:00, 11.82it/s]

CPU times: user 2min 50s, sys: 13.3 s, total: 3min 3s
Wall time: 2min 48s


In [120]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09


Результат немного улучшился при увеличении окна с 10 до 30, однако безлайн побить все равно не удалось.

Еще я попробовала не считать слишком короткие слова и стоп-слова (кусочек с len_check), но результат стал хуже)

In [ ]:
keywords_rw = data['content_norm'].progress_apply(lambda x: get_kws(x, top = 10, window_size = 30, len_check = True))

In [139]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.09
Recall -  0.18
F1 -  0.12
Jaccard -  0.07


6. Графы с networkx

In [140]:
import networkx as nx

In [153]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    # менять тут 
    node2measure = dict(nx.pagerank_numpy(G)) #pagerank
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [162]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 1min 42s, sys: 1min 17s, total: 2min 59s
Wall time: 1min 32s


In [163]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Я уменьшила размер окна до 5: все метрики за исключением recall преодолели бейзлайн

In [154]:
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 5))

In [155]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1


Если уменьшить до 2, то precision еще возрастет, а остальные метрики ухудшатся

In [156]:
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 2))

In [157]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.24
Recall -  0.1
F1 -  0.13
Jaccard -  0.08


Если, наоборот, окно увеличить, тогда recall возрастет и побьет бейзлайн, но остальные метрики будут небольшими

In [160]:
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 15))

In [161]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.1
Recall -  0.29
F1 -  0.14
Jaccard -  0.08


## Домашнее задание

В семинаре использовался только небольшой кусочек данных. На всех данных пересчитайте baseline (tfidf). 

**Ваша задача - предложить 3 способа побить бейзлайн на всех данных.**

Нет никаких ограничений кроме:

1) нельзя изменять метрику  
2) решение должно быть воспроизводимым  
3) способы дожны отличаться друг от друга не только гиперпараметрами (например, нельзя три раза поменять гиперпарамтры в TfidfVectorizer и сдать работу)  
4) изменение количества извлекаемых слов не является улучшением (выберите одно значение и используйте только его)  

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - 

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

В поисках идей можно почитать обзоры по теме (посмотрите еще статьи, в которых цитируются эти обзоры): https://www.semanticscholar.org/search?year%5B0%5D=2012&year%5B1%5D=2020&publicationType%5B0%5D=Reviews&q=keyword%20extraction&sort=relevance

**Использовать доступные готовые решения тоже можно**. Так что погуглите перед тем, как приступать к заданию. 